# Get-RscMssqlDatabaseRecoveryPoint
When doing any type of recovery operation, restore, export or live mount, you will need to provide a point in time to recover to. Rubrik expects a timesamp in milliseconds as the recovery point. This can be hard to define on your own, so 
Get-RscMssqlDatabaseRecoveryPoint allows for the easy to get that value. 

Get-RscMssqlDatabaseRecoveryPoint accepts 2 parameters, a database and a point in time value. 

For the point in time value, there are a few different possibilities
- **latest**: This will tell Rubrik to export the database to the latest recovery point Rubrik knows about. This will include the last full and any logs to get to the latest recovery point
- **lastfull**: This will tell Rubrik to restore back to the last full backup it has
- **RestoreTime**: This can be done in 1 of 2 formats
    - Format: (HH:MM:SS.mmm) or (HH:MM:SS.mmmZ) - Restores the database back to the local or UTC time (respectively) at the point in time specified within the last 24 hours
    - Format: Any valid <value> that PS Get-Date supports in: "Get-Date -Date <Value>"
        - Example: "2018-08-01T02:00:00.000Z" restores back to 2AM on August 1, 2018 UTC.

# Workflow
To get the recovery point of a MSSQL database, you need to do the following
1. Use **Connect-RSC** to Connect to RSC
1. Use **Get-RscCluster** to get the cluster id that the SQL Server is registered. This is akin to the datacenter that the SQL Server is located in. While you can run the next command without the cluster ID, you may get more records back than expected. This is because your SQL Server Host could exist on 2 different cluster. Once as a replicated source and once as the replicated target. 
1. Use **Get-RscHost** to get the id of the host. This will be used in the next cmdlet when looking up a SQL Instance
1. Use **Get-RscMssqlInstance** to get the instance id of the SQL Server based on the name of the instance and the cluster where it is located. 
1. Use **Get-RscMssqlDatabase** to get the database ID based on the database name and the SQL Server Instance. 
1. Use **Get-RscMssqlDatabaseRecoveryPoint** to get the database recovery point. 
1. Use **Disconnect-RSC** to disconnect from RSC
 new-rscmssqllogBackup -RscMssqlDatabase $mssqldatabase  

## Connect to RSC

In [1]:
Import-Module RubrikSecurityCloud
Connect-RSC


ClientId                                AuthenticationState SdkSchemaVersion ServerVersion
--------                                ------------------- ---------------- -------------
client|kB5og68XXrFlfXcvWhn3jlHdf1OKyJCB          AUTHORIZED v20240219-20     v20240219-20



## Get the Cluster ID

In [2]:
$ClusterName = "vault-r-london"
$RscCluster = Get-RscCluster -Name $ClusterName

## Get the Host ID

In [3]:
$HostName = "rp-sql19s-001.perf.rubrik.com"
$RscHost = Get-RscHost -Name $HostName -OsType Windows

## Get the Instance Information

In [4]:
$RscMssqlInstance = Get-RscMssqlInstance -RscHost $RscHost -RscCluster $RscCluster

## Get the Database Information

In [5]:
$DatabaseName = "AdventureWorks2019"
$RscMssqlDatabase = Get-RscMssqlDatabase -Name $DatabaseName  -RscMssqlInstance $RscMssqlInstance

## Get the latest recovery point

In [6]:
Get-RscMssqlDatabaseRecoveryPoint -RscMssqlDatabase $RscMssqlDatabase -Latest

2024-02-29T15:27:29.000Z


## Get the last full recovery point

In [7]:
Get-RscMssqlDatabaseRecoveryPoint -RscMssqlDatabase $RscMssqlDatabase -LastFull

2024-02-29T15:25:49.000Z


## Get a relative point in time in the last 24 hours

In [8]:
Get-RscMssqlDatabaseRecoveryPoint -RscMssqlDatabase $RscMssqlDatabase -RestoreTime "02:00"

2024-02-29T07:00:00.000Z


## Get a precise point in time based on local time

In [9]:
Get-RscMssqlDatabaseRecoveryPoint -RscMssqlDatabase $RscMssqlDatabase -RestoreTime "2023-10-02T23:00:25.000"

2023-10-03T03:00:25.000Z


## Get a precise point in time based on UTC time

In [10]:
Get-RscMssqlDatabaseRecoveryPoint -RscMssqlDatabase $RscMssqlDatabase -RestoreTime "2023-10-02T23:00:25.000Z"

2023-10-02T23:00:25.000Z


---

# Disconnect from RSC
It is important to have each script you create end by disconnecting from RSC. If you do not, then you will run into issues with token authentication exhaustion. RSC allows a Service Account to have 10 concurrent open connections to RSC. Additional requests will be denied until existing connections are closed. 

You can easily disconnect your connection by running the below statement. 

In [11]:
Disconnect-Rsc

The Rubrik Security Cloud session has been terminated.
